    #Python script to share which will enable the public extract the tweets/data directly from Twitter. We are not sharing the dehaydrated tweets due to sharing policy of Twitter ... 

In [1]:
import pandas as pd
import numpy as np
import tweepy
import json

    # load the file (using the correct path)

In [19]:
df = pd.read_csv('spd-dataset_01_ids.csv')
list_of_users_ids = df.UserID.tolist() # remember to separate the AccountClass during collection

In [26]:
len(df), len(list_of_users_ids)

(17515, 17515)

**Authentication with Twitter:**
    #to get authentication keys visit: https://developer.twitter.com/en/docs/basics/authentication/guides/access-tokens.html

In [23]:
# authentication keys:
consumer_key = "YOUR CONSUMER KEY"
consumer_secret = "YOUR CONSUMER SECRET"
access_token = "YOUR ACCESS TOKEN"
access_token_secret = "YOUR  ACCESS TOKEN SECRET"

In [3]:
# authentication instance:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) # required for validation
auth.set_access_token(access_token, access_token_secret) # required for access after validation of user
api =  tweepy.API(auth) # tweepy requires authenticated user to operate

    # search or look-up for users ... each search instance should not exceed 100, e.g list_of_users_ids[:100] 

In [ ]:
# empty dataframe to store extracted data:
dfs = pd.DataFrame()

# read the data in chunks of 100 ... 
for chunk in pd.read_csv('spd-dataset_01_ids.csv',chunksize=100):
    list_of_users_ids = [int(i) for i in chunk.UserID.tolist()]
    # users' look-up .... 
    users = api.lookup_users(user_ids=list_of_users_ids)
    
    # data structure to store relevant fields/metadata in the tweet object ... 
    extracts = {'UserID':[],'ScreenName':[],'AccountCreated':[],'Description':[],'Tweets':[],'TweetCreatedAt':[],\
                'TweetFavouriteCount':[], 'FollowersCount':[], 'StatusesCount':[],'FriendsCount':[]} 
    
    # extract relevant metadata ...
    for user in users:
        try: # avoids stopping incase of suspended/deleted account 
            extracts['UserID'].append(user.id)
            extracts['ScreenName'].append(user.screen_name)
            extracts['AccountCreated'].append(user.created_at)
            extracts['Description'].append(user.description)
            extracts['Tweets'].append(user.status.text)
            extracts['TweetCreatedAt'].append(user.status.created_at)
            extracts['TweetFavouriteCount'].append(user.status.favorite_count)
            extracts['FollowersCount'].append(user.followers_count)
            extracts['StatusesCount'].append(user.statuses_count)
            extracts['FriendsCount'].append(user.friends_count)
        except:
            continue
    
    # convert to dataframe for fruther processing .... 
    try:
        d = pd.DataFrame(extracts)
        dfs = dfs.append(d) # update the empty dataframe created earlier
    except:
        continue

    # save the extracted data:

In [ ]:
dfs.to_csv('spd-dataset_extracts.csv', index_label=False)

In [ ]:
# the extracted data should look similar to the dataframe below:

In [18]:
dfs

,UserID,ScreenName,AccountCreated,Description,Tweets,TweetCreatedAt,TweetFavouriteCount,FollowersCount,StatusesCount,FriendsCount
0,30925532,genna_buck,2009-04-13 18:39:12,Freelance journalist+editor with words in @Tor...,"RT @SherryAske: Imagine a MONTH-long, 7-stage ...",2019-03-25 02:02:33,0,2018,6374,1903
1,58211781,NalakaG,2009-07-19 14:54:11,"Science writer, tech watcher & media analyst. ...",RT @BillGates: I’m inspired by Azim Premji’s c...,2019-03-25 08:55:00,0,17984,32502,350
2,1038119899,GeoffFolsom,2012-12-26 21:28:08,Seeking news/sport story ideas at gfolsom@redm...,"@TEXPAMAN @Evan_P_Grant Yikes, this is worse t...",2019-03-25 13:13:24,0,721,22469,728
3,38382649,royaltyuso,2009-05-07 06:11:31,Global Citizen | @BangorUni alumnus - https://...,Thank you Lord JESUS for this afternoon.,2019-03-25 11:00:55,4,8686,81231,2077
4,76351820,louistheroux,2009-09-22 14:51:17,I am involved in the making of documentaries. ...,Good morning! https://t.co/NJeior3DfT,2019-03-24 07:57:19,3038,2005045,3163,1681
